In [1]:
import nltk.sentiment
import numpy as np
import pandas as pd
import sklearn.tree
import sklearn.linear_model
import sklearn.ensemble
import sklearn.neural_network
import sklearn.neighbors
import sklearn.discriminant_analysis
import sklearn.naive_bayes
import sklearn.svm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import math

C:\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
# Read in the data

# Must do it line by line due to error in Pandas' read_csv()
with open("c:/users/alex/documents/Sentiment Analysis Dataset.csv", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
lines = [line.strip().split(",", maxsplit = 3) for line in lines]

header = lines[0]
del lines[0]

data = pd.DataFrame(lines[:300000], columns = header)
del lines

In [4]:
# Normalize the text
repl = lambda m: re.sub(r'(.)\1+', r'\1\1', m) # Reduce instances of repeated characters     
data["FilteredText"] = data.SentimentText.str.lower().apply(repl)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
v = TfidfVectorizer(binary=True, min_df = 10, ngram_range=(1,2), use_idf=True) # A word must appear in at least 1000 tweets to be considered for tf-idf
x = v.fit_transform(data.FilteredText) # Calculate all tf-idf values for each word in every tweet

In [39]:
x

<300000x49805 sparse matrix of type '<class 'numpy.float64'>'
	with 5047343 stored elements in Compressed Sparse Row format>

In [6]:
X = pd.DataFrame(x.toarray()) # Reformat

In [14]:
from sklearn.model_selection import cross_val_score, GridSearchCV

In [8]:
sid = SentimentIntensityAnalyzer()
outputs = data.FilteredText.apply(lambda x: sid.polarity_scores(x)["compound"])

In [ ]:
newX = pd.concat([X, outputs], axis = 1)


In [14]:
newX = newX.to_sparse()

In [9]:
# Do a grid search to find the best hyperparameters for the Random Forest

#model = sklearn.ensemble.RandomForestClassifier()

#parameters = {
#    'n_estimators' : [1, 5, 10, 20, 30],
#    'max_depth' : [1, 5, 10, 20, 30]
#}
#searcher = GridSearchCV(model, parameters, cv = 3, scoring="accuracy", n_jobs = 6, verbose = 10000)
#searcher.fit(x, data.Sentiment)
#searcher.best_params_

In [15]:
model = sklearn.ensemble.RandomForestClassifier(max_depth = 30, n_estimators=20)
cross_val_score(model, x, data.Sentiment, cv = 5, verbose = 3, n_jobs = 5).mean()

[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  5.3min remaining:  7.9min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  5.5min finished


0.71642954737452258

In [40]:
## BEST MODEL SO FAR
model2 = sklearn.linear_model.LogisticRegression()
cross_val_score(model2, x, data.Sentiment, cv = 5, verbose = 3, n_jobs = 5).mean()

[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    7.0s remaining:   10.5s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    9.2s finished


0.79414335462100927

In [29]:
model2 = sklearn.linear_model.LogisticRegression()
cross_val_score(model2, x, data.Sentiment, cv = 5, verbose = 3, n_jobs=5).mean()

[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    7.1s remaining:   10.6s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    9.2s finished


0.79414335462100927

In [44]:
model2.fit(x, data.Sentiment)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
model2.coef_

array([[ 1.00632834,  0.78149828, -0.3849862 , ...,  0.99888723,
         0.19634944,  1.08569498]])

In [1]:
# Commented out because SVM is too computationally heavy for this size of the training data
#model3 = sklearn.svm.LinearSVC()
#cross_val_score(model3, x, data.Sentiment, cv = 5, verbose = 3, n_jobs = 5).mean()

NameError: name 'sklearn' is not defined

In [ ]:
model4 = sklearn.naive_bayes.GaussianNB()
cross_val_score(model4, X, data.Sentiment, cv = 5, verbose = 3).mean()

[CV]  ................................................................


In [ ]:
model5 = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis()
cross_val_score(model5, x, data.Sentiment, cv = 5, verbose = 3).mean()

In [ ]:
model6 = sklearn.neighbors.KNeighborsClassifier()
cross_val_score(model6, x, data.Sentiment, cv = 5, verbose = 3).mean()

In [ ]:
model7 = sklearn.neural_network.MLPClassifier(alpha = 1)
cross_val_score(model7, x, data.Sentiment, cv = 5, verbose = 3).mean()

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [9]:
sid = SentimentIntensityAnalyzer()

In [33]:
((((check > 0) * 1) == data.iloc[:100000, :].Sentiment.astype('int')) * 1).mean()

0.64431000000000005

In [31]:
data.Sentiment.astype('int').mean()

0.5005520620133825

In [37]:
outputs[:10].as_matrix()

array([-0.5256, -0.296 , -0.1027, -0.4767, -0.6351, -0.4404,  0.    ,
        0.2732, -0.1531,  0.0772])

In [ ]:
2+2